# ***Set Parameters to define model etc***

In [ ]:
trancuate_t = 96
#### Undersampling Strategy:#####
Undersampling = "Regular" # Options: Regular or Possoin or Complementary_Masks
Sampling_Mask = "Complementary_Masks" #Options: Single_Combination or One_Mask or Complementary_Masks
AF = 4 #  acceleration factor

#### Model Input and Output ####
GT_Data = "LowRank" # Options: FullRank LowRank for GROUNDTRUTH!
Low_Rank_Input = True ## apply low rank to the input as well if True

####M Model Parameters ####
batch_size=4096
num_convs = 2

#### Parameters for which axes to choose as Model input and output
DOMAIN = "xz"  # Input axes that goes into the network (as well as output), so far fT = frequeny and time, tT, xy nd xz are valid options

In [2]:
import os
import sys

sys.path.append('../scripts')
sys.path.append('../models')

os.environ["CUDA_VISIBLE_DEVICES"]= '1' #, this way I would choose GPU 3 to do the work

import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import zoom # for compressing images / only for testing purposes to speed up NN training
from torch.utils.data import DataLoader, Subset
import torch.optim as optim
import torch.nn as nn
from data_preparation import *
from data_undersampling import *
from output_statistics import *

from interlacer_layer_modified import *
from Residual_Interlacer_modified import *
from skimage.metrics import structural_similarity as ssim 

#### Model import correct model
from Naive_CNN_2D import * #from Naive_CNN_3D_Residual_No_Batch_Norm import *

grouped_time_steps = 1 # Set how many subsequent time steps you want to give to the network at once. Values allowed: 1, 2, 4, 8 (because it has to divide 8)

In [3]:
#### Define ground truth path####
if GT_Data == "FullRank":
    ground_truth_path = "../data/Ground_Truth/Full_Rank/P03-P08_truncated_k_space.npy"
elif GT_Data == "LowRank":
    ground_truth_path = "../data/Ground_Truth/Low_Rank/LR_8_P03-P08_self.npy"

#### Assemble saving path ####

#### Definie Model path
if GT_Data == "FullRank":
    saved_model_path = f"../saved_models/Naive_CNN_2D/"+DOMAIN+"/Full2Full/"+Undersampling+f'/AF_{AF}/'+f'Truncate_t_{trancuate_t}/'+Sampling_Mask+f'/{num_convs}Layer/best_model.pth'
elif GT_Data == "LowRank":
    saved_model_path = f"../saved_models/Naive_CNN_2D/"+DOMAIN+"/Low2Low/"+Undersampling+f'/AF_{AF}/'+f'Truncate_t_{trancuate_t}/'+Sampling_Mask+f'/{num_convs}Layer/best_model.pth'

#### Define Input Data path
undersampled_data_path = "../data/Undersampled_Data/"+Undersampling+f'/AF_{AF}/'+Sampling_Mask+'/data.npy'

if not os.path.exists(saved_model_path):
    raise FileNotFoundError(f"Error: The path '{saved_model_path}' does not exist.")

#### load data
Ground_Truth = np.load(ground_truth_path)
Undersampled_Data = np.load(undersampled_data_path)

#### additionally make LowRank 8 transformation on input of network, this improves the error significantly!
if Low_Rank_Input:
    Undersampled_Data[...,0] = low_rank(Undersampled_Data[...,0], 8)
    Undersampled_Data[...,1] = low_rank(Undersampled_Data[...,1], 8)
    Undersampled_Data[...,2] = low_rank(Undersampled_Data[...,2], 8)
    Undersampled_Data[...,3] = low_rank(Undersampled_Data[...,3], 8)
    Undersampled_Data[...,4] = low_rank(Undersampled_Data[...,4], 8)
    Undersampled_Data[...,5] = low_rank(Undersampled_Data[...,5], 8)
    
#### trancuate spectral dimension with trancuate_t
Ground_Truth, Undersampled_Data = Ground_Truth[:,:,:,:trancuate_t,:,:], Undersampled_Data[:,:,:,:trancuate_t,:,:]    
    
####  Prepare data for appropriate domain    
if DOMAIN == "fT":    
    Undersampled_Data = np.fft.fftshift(np.fft.fft(Undersampled_Data, axis=-3), axes=-3)
    Ground_Truth = np.fft.fftshift(np.fft.fft(Ground_Truth, axis=-3), axes=-3) 
    
    # bring fT to the first two indices
    reshape_vector = (3, 4, 0, 1, 2, 5)
    inverse_reshape = (2, 3, 4, 0, 1, 5)
    
elif DOMAIN == "tT":    
    # bring fT to the first two indices
    reshape_vector = (3, 4, 0, 1, 2, 5)
    inverse_reshape = (2, 3, 4, 0, 1, 5)
    
elif DOMAIN == "xy":    
    # bring fT to the first two indices
    reshape_vector = (0, 1, 2, 3, 4, 5)
    inverse_reshape = (0, 1, 2, 3, 4, 5)
    
elif DOMAIN == "xz":    
    # bring fT to the first two indices
    reshape_vector = (0, 2, 1, 3, 4, 5)
    inverse_reshape = (0, 2, 1, 3, 4, 5)    
    
else: 
    raise Exception("No valid Domain chosen!")

    
Undersampled_Data, Ground_Truth = Undersampled_Data.transpose(reshape_vector) , Ground_Truth.transpose(reshape_vector) 
original_shape = Ground_Truth.copy().shape

FileNotFoundError: Error: The path '../saved_models/Naive_CNN_2D/xz/Low2Low/Regular/AF_4/Truncate_t_96/Complementary_Masks/2Layer/best_model.pth' does not exist.

In [ ]:
ground_truth = Ground_Truth

#### Assign undersampled network input ####
NN_input = Undersampled_Data

# #### Collapse ununsed dimensions ####
ground_truth = ground_truth.reshape(Ground_Truth.shape[0], Ground_Truth.shape[1], -1)
NN_input = NN_input.reshape(Ground_Truth.shape[0], Ground_Truth.shape[1], -1)

# # #### reshape for pytorch ####
data, labels  = reshape_for_pytorch_2D(NN_input), reshape_for_pytorch_2D(ground_truth)

In [ ]:
# Create TensorDataset instances
dataset = TensorDataset(data, labels)

# Create DataLoaders
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  # Number of convolutional layers
model = Naive_CNN_2D(in_channels=2, num_convs=num_convs).to(device)

checkpoint = torch.load(saved_model_path, map_location=device)
# Extract the model state_dict
model.load_state_dict(checkpoint["model_state_dict"])

In [ ]:
Test_Set = 0

Model_Outputs_Test_Set = evaluate_model_for_predictions(model, data_loader, device, original_shape, inverse_reshape)[..., Test_Set]

if DOMAIN == "fT":
    Model_Outputs_Test_Set = np.fft.ifft(np.fft.ifftshift(Model_Outputs_Test_Set, axes=-2), axis=-2)

Undersampled_Data = np.load(undersampled_data_path)

Undersampled_Data[...,0] = low_rank(Undersampled_Data[...,0], 8)

Ground_Truth = np.load(ground_truth_path)
ground_truth = Ground_Truth[:,:,:,:trancuate_t,:,0]

mask = np.load('../data/masks.npy')
mask_5 = mask[:,:,:,0]
mask_5D = mask_5[:,:,:, np.newaxis, np.newaxis]

Model_Outputs_Test_Set = Model_Outputs_Test_Set*mask_5D
ground_truth = ground_truth*mask_5D
norm_values_test = np.ones((trancuate_t,8))
### to be fair, I compute a LR approximation, as a trivial baseline comparison + apply the mask
model_input = Undersampled_Data[...,0]*mask_5D
model_input = model_input[:,:,:,:trancuate_t,:]

plot_general_statistics(Model_Outputs_Test_Set, model_input, ground_truth, trancuate_t, norm_values_test, label = "Model Output", label2 = "Model Input")
plot_general_statistics_PSNR(Model_Outputs_Test_Set, model_input, ground_truth, trancuate_t, norm_values_test, label = "Model Output", label2 = "Model Input")
plot_general_statistics_SSIM(Model_Outputs_Test_Set, model_input, ground_truth, trancuate_t, norm_values_test, label = "Model Output", label2 = "Model Input")

In [ ]:
tf = 50 #15 = Water, 17 Glucose, 23 Glx
T = 7

ground_truth = ground_truth*mask_5D
comparison_Plot_3D_vs_Ifft(Model_Outputs_Test_Set, ground_truth, model_input, tf, T, domain="spectral")

In [ ]:
Model_Outputs_Test_Set, ground_truth =  evaluate_model_for_predictions(model, test_loader, device, trancuate_t), Ground_Truth[:,:,:,:trancuate_t,:,0]

mask = np.load('../data/masks.npy')
mask_5 = mask[:,:,:,0]
mask_5D = mask_5[:,:,:, np.newaxis, np.newaxis]

Model_Outputs_Test_Set = Model_Outputs_Test_Set*mask_5D

norm_values_test = norm_values_test.reshape(trancuate_t,8)
model_input = Undersampled_Data[:,:,:,:trancuate_t,:,0]
ground_truth = ground_truth*mask_5D
### to be fair, I compute a LR approximation, as a trivial baseline comparison + apply the mask
model_input = low_rank(model_input, 8)*mask_5D

ground_truth_spectral = np.fft.fftshift(np.fft.fft(ground_truth, axis=-2), axes=-2)
model_pred_spectral = np.fft.fftshift(np.fft.fft(Model_Outputs_Test_Set, axis=-2), axes=-2)
IFF_LR_8 = np.fft.fftshift(np.fft.fft(model_input, axis=-2), axes=-2)


In [ ]:
model_input = Undersampled_Data[:,:,:,:trancuate_t,:,0]
model_input = low_rank(model_input, 8)*mask_5D

IFF_LR_8 = np.fft.fftshift(np.fft.fft(model_input, axis=-2), axes=-2)
# Fixed indices for x, y, and T:
# ----------------------------------------------------------------------
# Fixed indices for x, y, and T
# ----------------------------------------------------------------------
x_fixed = 10
y_fixed = 15
T_fixed = 7

num_z = 21  # Number of z slices

# ----------------------------------------------------------------------
# Create a figure with 2 columns:
#   Column 1 -> Absolute value plots
#   Column 2 -> Residuals (GT - [Others])
# ----------------------------------------------------------------------
fig, axes = plt.subplots(nrows=num_z, ncols=2, figsize=(16, num_z * 3.5), sharex=True)

# If there's only one row, ensure axes is 2D
if num_z == 1:
    axes = np.array([axes])

for z in range(num_z):
    # Extract the spectra at [x_fixed, y_fixed, z, :, T_fixed]
    gt_spec  = ground_truth_spectral[x_fixed, y_fixed, z, :, T_fixed]
    mp_spec  = model_pred_spectral[x_fixed, y_fixed, z, :, T_fixed]
    iff_spec = IFF_LR_8[x_fixed, y_fixed, z, :, T_fixed]
    
    # ------------------------------------------------------------------
    # Column 1: Plot absolute spectra
    # ------------------------------------------------------------------
    ax_abs = axes[z, 0]
    ax_abs.plot(np.abs(gt_spec),  label='Ground Truth',    color='blue')
    ax_abs.plot(np.abs(mp_spec),  label='Model Prediction',color='orange')
    ax_abs.plot(np.abs(iff_spec), label='IFFT + LR 8',     color='green')
    ax_abs.set_ylim(-50000, 300000)
    # Row label on the Y-axis
    ax_abs.set_ylabel(f'z = {z}')
    
    if z == 0:
        ax_abs.set_title('Absolute Value')
    ax_abs.legend(loc='upper left')
    
    # ------------------------------------------------------------------
    # Column 2: Plot residuals (Ground Truth - [Others])
    # ------------------------------------------------------------------
    ax_res = axes[z, 1]
    
    residual_gt_mp  = np.abs(gt_spec) - np.abs(mp_spec)
    residual_gt_iff = np.abs(gt_spec) - np.abs(iff_spec)
    
    ax_res.plot(residual_gt_mp,  label='GroundTruth - Model',        color='orange')
    ax_res.plot(residual_gt_iff, label='GroundTruth - IFFT + LR 8',  color='green')

    if z == 0:
        ax_res.set_title('Residual')
    ax_res.legend(loc='upper left')
    ax_res.set_ylim(-50000, 300000)
plt.tight_layout()
plt.show()

